In [1]:
#importing required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt #visualization
%matplotlib inline 
#sns.set(color_codes=True)

In [2]:
#importing train datasets
df_train= pd.read_csv('C:/Users/tamak/Desktop/Data Science/Predicts/EDSA_ZM-Team1/train_set.csv')
#importing test datasets
df_test = pd.read_csv('C:/Users/tamak/Desktop/Data Science/Predicts/EDSA_ZM-Team1/test_set.csv')

In [3]:
#creating a copy of train to work on so as to not overwrite origional train set data - only for golden apples
model_train2 = df_train.loc[df_train['Commodities'] == 'APPLE GOLDEN DELICIOUS'].copy()

model_train2 = model_train2.drop('Commodities',axis=1)
model_train2 = model_train2.drop('Province',axis=1)
model_train2 = model_train2.drop('Size_Grade',axis=1)
model_train2 = model_train2.drop('Container',axis=1)
model_train2 = model_train2.drop('Stock_On_Hand',axis=1)
model_train2 = model_train2.drop('Total_Qty_Sold',axis=1)
model_train2 = model_train2.drop('Weight_Kg',axis=1)
model_train2 = model_train2.drop('Date',axis=1)

#creating a copy of test to work on so as to not overwrite origional test set data - only for golden apples
model_test2 = df_test.loc[df_test['Commodities'] == 'APPLE GOLDEN DELICIOUS'].copy()

model_test2 = model_test2.drop('Commodities',axis=1)
model_test2 = model_test2.drop('Province',axis=1)
model_test2 = model_test2.drop('Size_Grade',axis=1)
model_test2 = model_test2.drop('Container',axis=1)
model_test2 = model_test2.drop('Stock_On_Hand',axis=1)
model_test2 = model_test2.drop('Total_Qty_Sold',axis=1)
model_test2 = model_test2.drop('Weight_Kg',axis=1)
model_test2 = model_test2.drop('Date',axis=1)
model_test2 = model_test2.drop('Index', axis=1)

In [4]:
#importing libraries from sklearn
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split as tts
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn import metrics
import math

y = model_train2.avg_price_per_kg # defining label
x = model_train2.drop('avg_price_per_kg',axis=1) # defining features

x_train, x_test, y_train, y_test = tts(x, y, test_size=0.3505, random_state=1) # creating train-test-split

model_lr = LinearRegression(fit_intercept=True) # linear regression model
model_lr.fit(x_train, y_train) 

model_las = Lasso(alpha=1.0) # lasso model
model_las.fit(x_train, y_train)

model_ridge = Ridge(alpha=1.0) # ridge model
model_ridge.fit(x_train, y_train)

Ridge()

In [5]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error as MSE

# Our forest consists of 100 trees with a max depth of 5 in this example
model_rf = RandomForestRegressor(n_estimators=100, max_depth=5)
model_rf.fit(x_train,y_train)

RandomForestRegressor(max_depth=5)

In [6]:
# Fit regression decision tree model
from sklearn.tree import DecisionTreeRegressor
model_dt = DecisionTreeRegressor(max_depth=5)
model_dt.fit(x_train,y_train)

DecisionTreeRegressor(max_depth=5)

In [7]:
from sklearn.ensemble import VotingRegressor

# Define the models which we'll include in our ensemble. 

models = [("LR",model_lr),("DT",model_dt),("LAS",model_las),("RDG", model_ridge)]

# Specify weights for weighted model averaging
model_weightings = np.array([0.1,0.3,0.3,0.3])
v_reg = VotingRegressor(estimators=models,weights=model_weightings)
v_reg.fit(x_train,y_train)

VotingRegressor(estimators=[('LR', LinearRegression()),
                            ('DT', DecisionTreeRegressor(max_depth=5)),
                            ('LAS', Lasso()), ('RDG', Ridge())],
                weights=array([0.1, 0.3, 0.3, 0.3]))

In [8]:
from sklearn.ensemble import StackingRegressor
 
meta_learner_reg = LinearRegression()
s_reg = StackingRegressor(estimators=models, final_estimator=meta_learner_reg)
s_reg.fit(x_train,y_train)

StackingRegressor(estimators=[('LR', LinearRegression()),
                              ('DT', DecisionTreeRegressor(max_depth=5)),
                              ('LAS', Lasso()), ('RDG', Ridge())],
                  final_estimator=LinearRegression())

In [9]:
from sklearn.ensemble import BaggingRegressor
# Instantiate BaggingRegressor model with a decision tree as the base model
bag_reg = BaggingRegressor(base_estimator = model_dt)
bag_reg.fit(x_train,y_train)

BaggingRegressor(base_estimator=DecisionTreeRegressor(max_depth=5))

In [10]:
from sklearn.ensemble import AdaBoostRegressor

bst_reg = AdaBoostRegressor(base_estimator = model_dt)
bst_reg.fit(x_train,y_train)

AdaBoostRegressor(base_estimator=DecisionTreeRegressor(max_depth=5))

In [19]:
# Define params_rf
params_rf = {'n_estimators':[100,350,500],'max_features':['log2','auto','sqrt', 0.5],'min_samples_leaf':[2,10,30]}
# Import GridSearchCV
from sklearn.model_selection import GridSearchCV
# Import RandomForestRegressor
from sklearn.ensemble import RandomForestRegressor

# Instantiate rf
rf = RandomForestRegressor(n_estimators=25,
            random_state=2)

# Instantiate grid_rf
grid_rf = GridSearchCV(estimator=rf,
                       param_grid=params_rf,
                       scoring='neg_mean_squared_error',
                       cv=10,
                       verbose=1,
                       n_jobs=-1)

grid_rf.fit(x_train, y_train)
# Extract the best estimator
best_model = grid_rf.best_estimator_

# Predict test set labels
y_pred = best_model.predict(x_test)


Fitting 10 folds for each of 36 candidates, totalling 360 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   30.6s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  2.7min
[Parallel(n_jobs=-1)]: Done 360 out of 360 | elapsed:  4.7min finished


In [20]:
# dictionary of results
results_dict = {'Training MSE':
                    {
                        "Linear": metrics.mean_squared_error(y_train, model_lr.predict(x_train)),
                        "Lasso": metrics.mean_squared_error(y_train, model_las.predict(x_train)),
                        "Ridge": metrics.mean_squared_error(y_train, model_ridge.predict(x_train)),
                        "Random Forest": metrics.mean_squared_error(y_train, model_rf.predict(x_train)),
                        "Decision Tree": metrics.mean_squared_error(y_train, model_dt.predict(x_train)),
                        "Voting": metrics.mean_squared_error(y_train, v_reg.predict(x_train)),
                        "Stacking": metrics.mean_squared_error(y_train, s_reg.predict(x_train)),
                        "Bagging": metrics.mean_squared_error(y_train, bag_reg.predict(x_train)),
                        "Ada Boost": metrics.mean_squared_error(y_train, bst_reg.predict(x_train)),
                        "Grid Search Best Model": metrics.mean_squared_error(y_train, best_model.predict(x_train)),
                    },
                'Test MSE':
                    {
                        "Linear": metrics.mean_squared_error(y_test, model_lr.predict(x_test)),
                        "Lasso": metrics.mean_squared_error(y_test, model_las.predict(x_test)),
                        "Ridge": metrics.mean_squared_error(y_test, model_ridge.predict(x_test)),
                        "Random Forest": metrics.mean_squared_error(y_test, model_rf.predict(x_test)),
                        "Decision Tree": metrics.mean_squared_error(y_test, model_dt.predict(x_test)),
                        "Voting": metrics.mean_squared_error(y_test, v_reg.predict(x_test)),
                        "Stacking": metrics.mean_squared_error(y_test, s_reg.predict(x_test)),
                        "Bagging": metrics.mean_squared_error(y_test, bag_reg.predict(x_test)),
                        "Ada Boost": metrics.mean_squared_error(y_test, bst_reg.predict(x_test)),
                        "Grid Search Best Model": metrics.mean_squared_error(y_test, best_model.predict(x_test)),
                    },
                'Test RMSE':
                    {
                        "Linear": math.sqrt(metrics.mean_squared_error(y_test, model_lr.predict(x_test))),
                        "Lasso": math.sqrt(metrics.mean_squared_error(y_test, model_las.predict(x_test))),
                        "Ridge": math.sqrt(metrics.mean_squared_error(y_test, model_ridge.predict(x_test))),
                        "Random Forest": math.sqrt(metrics.mean_squared_error(y_test, model_rf.predict(x_test))),
                        "Decision Tree": math.sqrt(metrics.mean_squared_error(y_test, model_dt.predict(x_test))),
                        "Voting": math.sqrt(metrics.mean_squared_error(y_test, v_reg.predict(x_test))),
                        "Stacking": math.sqrt(metrics.mean_squared_error(y_test, s_reg.predict(x_test))),
                        "Bagging": math.sqrt(metrics.mean_squared_error(y_test, bag_reg.predict(x_test))),
                        "Ada Boost": math.sqrt(metrics.mean_squared_error(y_test, bst_reg.predict(x_test))),
                        "Grid Search Best Model": math.sqrt(metrics.mean_squared_error(y_test, best_model.predict(x_test))),
                    },
                'R-Squared':
                    {
                        "Linear": metrics.r2_score(y_test, model_lr.predict(x_test)),
                        "Lasso": metrics.r2_score(y_test, model_las.predict(x_test)),
                        "Ridge": metrics.r2_score(y_test, model_ridge.predict(x_test)),
                        "Random Forest": metrics.r2_score(y_test, model_rf.predict(x_test)),
                        "Decision Tree": metrics.r2_score(y_test, model_dt.predict(x_test)),
                        "Voting": metrics.r2_score(y_test, v_reg.predict(x_test)),
                        "Stacking": metrics.r2_score(y_test, s_reg.predict(x_test)),
                        "Bagging": metrics.r2_score(y_test, bag_reg.predict(x_test)),
                        "Ada Boost": metrics.r2_score(y_test, bst_reg.predict(x_test)),
                        "Grid Search Best Model": metrics.r2_score(y_test, best_model.predict(x_test)),
                    }
                }

# create dataframe from dictionary
results_df = pd.DataFrame(data=results_dict)

results_df

,Training MSE,Test MSE,Test RMSE,R-Squared
Linear,4.233436,5.282723,2.298418,-0.166192
Lasso,4.233025,5.280997,2.298042,-0.165811
Ridge,4.233436,5.282723,2.298418,-0.166192
Random Forest,0.926920,8.192185,2.862199,-0.808472
Decision Tree,1.053688,8.495962,2.914783,-0.875533
Voting,2.521630,5.580657,2.362341,-0.231962
Stacking,1.011996,8.381113,2.895015,-0.850179
Bagging,0.950452,8.175599,2.859300,-0.804811
Ada Boost,0.843844,8.151859,2.855146,-0.799570
Grid Search Best Model,0.246973,8.413874,2.900668,-0.857411


In [13]:
y_train = model_train2.avg_price_per_kg # defining label
x_train = model_train2.drop('avg_price_per_kg',axis=1) # defining features

x_test = model_test2

y_pred = best_model.predict(x_test)

In [14]:
index = df_test['Index']

In [15]:
submission = pd.DataFrame({'Index' : index, 'avg_price_per_kg' : y_pred})

In [16]:
submission.to_csv(r'C:\Users\tamak\Desktop\Data Science\Predicts\EDSA_ZM-Team1\KaggleCSV.csv', index = False)